In [ ]:
# top-level requirements: pytorch-forecasting==0.10.3, citylearn==1.7, ipykernel, jupyter, tensorboard, plotly
# install numba for improved speed
# to use cuda support, run: pip3 install torch==1.13 --index-url https://download.pytorch.org/whl/cu117 - see https://pytorch.org/get-started/locally/

In [ ]:
import os
import json
from models.TFT.TFT_predictor import TFT_Predictor

In [ ]:
train_path = os.path.join('data','example','train')
val_path = os.path.join('data','example','validate')
with open(os.path.join(train_path,'metadata_ext.json')) as json_file:
    UCam_ids = json.load(json_file)["UCam_building_ids"]

---

In [ ]:
TFT_pred = TFT_Predictor(model_group_name='test',load=False)

Skip this

---

In [ ]:
model_type = 'load'
model_name = 'load_test'
i = 0
train_ds,val_ds = TFT_pred.format_CityLearn_datasets([train_path,val_path], model_type=model_type, building_index=i)
tft_model = TFT_pred.new_model(model_name,model_type,train_ds)

In [ ]:
TFT_pred.train_model(model_name,model_type,train_ds,val_ds)

---

Try loading the model group

In [ ]:
TFT_pred = TFT_Predictor(model_group_name='test',load='group')

---

In [ ]:
# create and train load models
model_type = 'load'
for i,id in enumerate(UCam_ids):
    model_name = f'{model_type}_{id}'
    train_ds,val_ds = TFT_pred.format_CityLearn_datasets([train_path,val_path], model_type=model_type, building_index=i)
    tft_model = TFT_pred.new_model(model_name,model_type,train_ds)
    TFT_pred.train_model(model_name,model_type,train_ds,val_ds)

In [ ]:
# create and train solar models
model_type = 'solar'
for i,id in enumerate(UCam_ids):
    model_name = f'{model_type}_{id}'
    train_ds,val_ds = TFT_pred.format_CityLearn_datasets([train_path,val_path], model_type=model_type, building_index=i)
    tft_model = TFT_pred.new_model(model_name,model_type,train_ds)
    TFT_pred.train_model(model_name,model_type,train_ds,val_ds)

In [ ]:
# create and train pricing model
model_type = 'pricing'
model_name = 'price'
train_ds,val_ds = TFT_pred.format_CityLearn_datasets([train_path,val_path], model_type=model_type)
tft_model = TFT_pred.new_model(model_name,model_type,train_ds)
TFT_pred.train_model(model_name,model_type,train_ds,val_ds)

In [ ]:
# create and train carbon model
model_type = 'carbon'
model_name = 'carbon'
train_ds,val_ds = TFT_pred.format_CityLearn_datasets([train_path,val_path], model_type=model_type)
tft_model = TFT_pred.new_model(model_name,model_type,train_ds)
TFT_pred.train_model(model_name,model_type,train_ds,val_ds)

In [ ]:
from citylearn.citylearn import CityLearnEnv
env = CityLearnEnv(os.path.join(val_path,'schema.json'))
t = 7720